In [56]:
from database.team_stats import TeamStatsManager
import pandas as pd

In [57]:
file_path = 'data/jornadas/2025/result_season_1_week_1.xlsm'

Este bloque actualiza la tabla de resultados de equipos

In [65]:
# Lee team resultados. Tabla 1
df_resultado = pd.read_excel(file_path, sheet_name='Partido', skiprows=3, usecols='B:G')
df_resultado.columns = ['local', 'vs', 'away', 'local_goals', '-', 'away_goals']

# Encontramos el inicio de la tabla stats
idx_stats = df_resultado[df_resultado['vs'] == 'Equipo'].index
idx_stats = idx_stats[0] + 4

df_resultado = df_resultado.drop(['vs', '-'], axis=1)
df_resultado = df_resultado.dropna()
#to int
df_resultado['local_goals'] = df_resultado['local_goals'].astype(int)
df_resultado['away_goals'] = df_resultado['away_goals'].astype(int)
df_resultado

,local,away,local_goals,away_goals
0,Amarillo,Rojo,2,0
1,Amarillo,Azul,1,0
2,Amarillo,Rojo,0,1
3,Rojo,Azul,1,0
4,Rojo,Amarillo,1,2
5,Azul,Amarillo,1,2
6,Rojo,Amarillo,2,1
7,Rojo,Azul,1,0
8,Rojo,Amarillo,2,1
9,Azul,Amarillo,2,1


In [67]:
# Get data stats from results
df_stats = df_resultado.copy()
teams = df_stats['local'].unique()
# add columns
for team in teams:
    df_stats[team] = 0

df_stats['local_points'] = df_stats.apply(
    lambda x: 3 if x['local_goals'] > x['away_goals']
        else 1 if x['local_goals'] == x['away_goals'] else 0, axis=1)
df_stats['away_points'] = df_stats.apply(
    lambda x: 3 if x['away_goals'] > x['local_goals']
        else 1 if x['away_goals'] == x['local_goals'] else 0, axis=1)

# Rename
local_stats = df_stats[['local', 'local_goals', 'away_goals', 'local_points']].rename(
    columns={'local': 'team', 'local_goals': 'goals', 'away_goals': 'goals_against', 'local_points': 'points'})
away_stats = df_stats[['away', 'away_goals', 'local_goals', 'away_points']].rename(
    columns={'away': 'team', 'away_goals': 'goals', 'local_goals': 'goals_against', 'away_points': 'points'})

df_stats = pd.concat([local_stats, away_stats])
df_stats = df_stats.groupby('team').sum().reset_index()

df_stats


,team,goals,goals_against,points
0,Amarillo,12,12,13
1,Azul,3,8,3
2,Rojo,13,8,22


In [71]:
# Get players stats
df_player_stats = pd.read_excel(file_path, skiprows=3, sheet_name='Partido', usecols="L:O")
df_player_stats.columns = ['name', 'team', 'goals', 'assists']
df_player_stats.dropna(how='all', inplace=True, subset=['goals', 'assists'])
df_player_stats.fillna(0, inplace=True)

# to int
df_player_stats['goals'] = df_player_stats['goals'].astype(int)
df_player_stats['assists'] = df_player_stats['assists'].astype(int)

df_player_stats


,name,team,goals,assists
0,Samuel Rojas,Amarillo,2,2
1,Joao,Amarillo,6,2
2,Vicente,Amarillo,0,1
3,Diego,Rojo,5,1
4,Andres Ruiz,Rojo,1,2
5,Abu,Rojo,3,2
6,Andres Diaz,Rojo,2,1
7,Ricardo,Amarillo,1,0
8,Joel,Amarillo,1,2
9,Eddy,Azul,2,1


Este bloque actualiza la de estadísticas agregando los stats de la jornada desde el excel de jornada.
- name
- team
- goals
- assists

Se debe leer luego desde la app para recoger 
- media
- mvp
- votes
- total_votes 

 Actualizar

In [ ]:
# check if the week result already exists
update_week = True
if trm.get_week_result(year, season, week):
    update = input("The week result already exists. Do you want to update it? (y/n): ")
    if update.lower() == 'y':
        trm.delete_week_result(year, season, week)
    else:
        update_week = False

if update_week:
    for row in df_resultado.iterrows():
        trm.add_team_result(
            year=year,
            season=season,
            match_week=week,
            team_name=row[1]["Equipo"],
            goals=row[1]["A favor"],
            goals_against=row[1]["En contra"],
            points=row[1]["Puntos"]
    )

In [11]:
# Agregando las estadisticas de los jugadores
# Primero leemos el registro de jugadores
df_players = pd.read_excel(file_path, skiprows=1, nrows=28)
df_players = df_players.iloc[:, 1:].copy()
df_players.columns = ['Nombre', 'Equipo']
df_players.dropna(inplace=True, subset='Nombre', axis=0)

df_statistics = pd.read_excel(file_path, sheet_name='Partido', skiprows=3, usecols="L:O")
df_statistics.dropna(inplace=True, axis=0, how='all')
df_statistics.fillna(0, inplace=True)
df_statistics.drop(columns='Equipo', inplace=True)
df_statistics.columns = df_statistics.columns.str.strip()

df_excel = pd.merge(df_players, df_statistics, on='Nombre', how='outer')
df_excel.fillna(0, inplace=True)
df_excel.columns = ['name', 'team', 'goals', 'assists']


from database.players import PlayerManager
pm = PlayerManager()
fantasy_id_map = pm.get_fantasy_id()

df_excel['id'] = df_excel['name'].map(fantasy_id_map)
df_excel

c:\Users\Eddy.Tovar\Documents\Datos\personal\proyectos\fantasy_fulbo\venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,name,team,goals,assists,id
0,Abu,Azul,2.0,1.0,1
1,Andres Diaz,Azul,0.0,0.0,4
2,Andres Freitas,Amarillo,0.0,1.0,5
3,Andres Ruiz,Azul,0.0,0.0,7
4,Arturo,Rojo,2.0,0.0,10
5,Bravo,Amarillo,0.0,0.0,14
6,Carlos,Negro,0.0,1.0,15
7,Chino,Rojo,1.0,0.0,16
8,De Tovar,Amarillo,0.0,0.0,19
9,Della Invitado,Rojo,0.0,1.0,65


Falta por agregar el código que escrapea desde la app. Lo he hecho en otro notebook para poder recuperar el histórico.